# Lesson 3: Handling AI Interactions with the Chat Service Layer

# Handling AI Interactions with the Chat Service Layer

In the previous lesson, we explored the `ChatManager` class, which plays a crucial role in managing chat data within our application. Now, we will take the next step in our journey by building the **Chat Service Layer**. This layer is essential for integrating the language model with chat sessions, allowing us to process user messages and generate AI responses. By the end of this lesson, you will understand how to set up the `ChatService` class, create chat sessions, and process messages using OpenAI’s API.

The service layer acts as a bridge between the model layer (where data is managed) and the AI model (which generates responses). It is responsible for orchestrating the flow of data and ensuring that user interactions are handled smoothly. Let’s dive into the details of setting up this important component.

---

## Setting Up the `ChatService` Class

The `ChatService` class is the heart of our service layer. It manages chat sessions and interacts with the OpenAI client to generate AI responses.

### Initialization

```python
import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        self.chat_manager   = ChatManager()
        self.openai_client  = OpenAI()
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')
```

- We instantiate `ChatManager` to handle chat data.
- We initialize the OpenAI client.
- We load the `system_prompt` from a file (using a helper method).

---

## Loading the System Prompt

The **system prompt** guides the AI’s behavior by providing context and instructions.

```python
def load_system_prompt(self, file_path: str) -> str:
    """Load the system prompt from file."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading system prompt: {e}")
        return "You are a helpful assistant."
```

- Reads the prompt from `file_path`.
- Falls back to a default prompt on error, ensuring continuity.

---

## Creating a New Chat Session

To start a conversation, we need a unique chat session.

```python
def create_chat(self, user_id: str) -> str:
    """Create a new chat session."""
    chat_id = str(uuid.uuid4())
    self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
    return chat_id
```

- Generates a `chat_id` using `uuid`.
- Calls `ChatManager.create_chat(...)` with `user_id`, `chat_id`, and `system_prompt`.

---

## Processing User Messages

The `process_message` method handles incoming user messages, requests an AI completion, and updates the chat history.

### Steps

1. **Retrieve the chat**  
   Use `get_chat`; throw an error if not found.  
2. **Add the user message**  
   Append to the chat history.  
3. **Generate AI response**  
   Send the full conversation (system prompt + messages) to OpenAI.  
4. **Store and return the AI response**  
   Append the assistant’s reply to the history.  
5. **Error handling**  
   Wrap the OpenAI call in `try/except` to surface errors gracefully.

### Implementation

```python
def process_message(self, user_id: str, chat_id: str, message: str) -> str:
    """Process a user message and get AI response."""
    
    # 1. Retrieve the chat
    chat = self.chat_manager.get_chat(user_id, chat_id)
    if not chat:
        raise ValueError("Chat not found")
    
    # 2. Add user message to history
    self.chat_manager.add_message(user_id, chat_id, "user", message)
    
    try:
        # 3. Get AI response
        conversation = self.chat_manager.get_conversation(user_id, chat_id)
        response = self.openai_client.chat.completions.create(
            model="gpt-4",
            messages=conversation,
            temperature=0.7,
            max_tokens=500
        )
        ai_message = response.choices[0].message.content
        
        # 4. Add AI response to history
        self.chat_manager.add_message(user_id, chat_id, "assistant", ai_message)
        return ai_message

    except Exception as e:
        # 5. Handle errors
        raise RuntimeError(f"Error getting AI response: {e}")
```

> **Note:** We set `temperature=0.7` for balanced creativity and `max_tokens=500` to allow detailed yet concise replies—ideal for a customer‑service context.

---

## Example: Simulating a Chat Session

Below is how you’d use `ChatService` in `main.py` to create a session and exchange a message.

```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID and create session
user_id = "user123"
chat_id  = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Simulate sending a messageHere’s the full implementation of `ChatService` with `create_chat`, plus a quick test:

```python
# services/chat_service.py

import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        # Initialize ChatManager to handle chat data
        self.chat_manager   = ChatManager()
        # Initialize OpenAI client (you can configure with api_key/env as needed)
        self.openai_client  = OpenAI()
        # Load the system prompt for all new chats
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """
        Create a new chat session for a given user.
        Returns the generated chat_id.
        """
        # 1. Generate a unique chat ID
        chat_id = str(uuid.uuid4())
        # 2. Use ChatManager to create and store the new chat
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        # 3. Return the new chat_id
        return chat_id
```

And here’s how you’d test it in a separate script (e.g. `main.py`):

```python
# main.py

from services.chat_service import ChatService

# 1) Initialize the chat service
chat_service = ChatService()

# 2) Define a sample user ID
user_id = "user123"

# 3) Create a new chat session and capture the chat_id
chat_id = chat_service.create_chat(user_id)

# 4) Print out the chat ID to verify
print(f"Chat session created with ID: {chat_id}")
```

**When you run `main.py`, you should see output similar to:**

```
Chat session created with ID: e3b0c442-98fc-4a5f-b13c-6d7f9b2a1234
```

This confirms that `create_chat` is generating a UUID, storing the session via `ChatManager`, and returning the new `chat_id`.
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")
```

**Sample output:**

```text
Chat session created with ID: 01a17870-8a4f-4b6f-a3ce-f04e1136d597
AI Response: Hello! I'm here to help with any questions or concerns you might have regarding our IT services. How can I assist you today?
```

---

## Summary and Next Steps

In this lesson, you learned how to:

- Set up the `ChatService` class and initialize its components  
- Load and fall back on a system prompt  
- Create unique chat sessions via `uuid`  
- Process user messages end‑to‑end with OpenAI’s API  
- Handle errors gracefully  

**Practice exercise:** Experiment by adding different `temperature` and `max_tokens` values, or extend `ChatService` with logging and metrics. Happy coding!  


## Implementing Chat Session Creation in ChatService

Let's begin by implementing the create_chat method within the ChatService class. This method is essential for generating a unique chat ID and initializing a chat session. Here's what you need to do:

Implement the create_chat method within the ChatService class.

Use the uuid module to generate a unique chat ID.
Utilize the ChatManager to create a chat session, associating it with the user ID, chat ID, and system prompt.
Ensure the method returns the generated chat ID.
Test the chat creation using the service.

Initialize an instance of ChatService.
Define a sample user ID to simulate a user.
Call the create_chat method with the user ID and store the returned chat ID.
Print the chat ID to verify the successful creation of the chat session.
This task will guide you through the process of creating and managing chat sessions, a fundamental aspect of building a chatbot service.


```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        # TODO: Initialize ChatManager to handle chat data
        self.chat_manager = 
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    # TODO: Define the create_chat method
    # - Parameters: user_id
    # - Generate a unique chat ID using uuid
    # - Use chat_manager to create a chat session with user_id, chat_id, and system_prompt
    # - Return the chat_id


```

```python
from services.chat_service import ChatService

# TODO: Initialize a ChatService instance

# TODO: Define a variable with a sample user identifier, e.g., "user123"

# TODO: Call the create_chat method with the user ID and store the chat ID

# TODO: Print the chat ID


```

Here’s the full implementation of `ChatService` with `create_chat`, plus a quick test:

```python
# services/chat_service.py

import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        # Initialize ChatManager to handle chat data
        self.chat_manager   = ChatManager()
        # Initialize OpenAI client (you can configure with api_key/env as needed)
        self.openai_client  = OpenAI()
        # Load the system prompt for all new chats
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """
        Create a new chat session for a given user.
        Returns the generated chat_id.
        """
        # 1. Generate a unique chat ID
        chat_id = str(uuid.uuid4())
        # 2. Use ChatManager to create and store the new chat
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        # 3. Return the new chat_id
        return chat_id
```

And here’s how you’d test it in a separate script (e.g. `main.py`):

```python
# main.py

from services.chat_service import ChatService

# 1) Initialize the chat service
chat_service = ChatService()

# 2) Define a sample user ID
user_id = "user123"

# 3) Create a new chat session and capture the chat_id
chat_id = chat_service.create_chat(user_id)

# 4) Print out the chat ID to verify
print(f"Chat session created with ID: {chat_id}")
```

**When you run `main.py`, you should see output similar to:**

```
Chat session created with ID: e3b0c442-98fc-4a5f-b13c-6d7f9b2a1234
```

This confirms that `create_chat` is generating a UUID, storing the session via `ChatManager`, and returning the new `chat_id`.

## Implementing Message Processing in Chat Service

## Enhancing AI Response Generation in Chat Service

## Enhancing AI Context Retention in Multi-Turn Conversations

## Managing Multiple Chat Sessions for a User